# O2E Brands Data Engineer Coding Assignment



In [32]:
import requests
import urllib.request
import pandas as pd
from bs4 import BeautifulSoup

## Part 1: Data Capture

In [29]:
def extract_paper(N):
    """
    This is function is to metadata on the latest N paper submissions from a website. 
    
    Arguments:
        N (int): the number of paper submissions
        
    Return:
        df (Pandas Dataframe): a dataframe with titles, description, authors and urls
        csv file: a csv file written in local
    """
    
    website_url = 'http://export.arxiv.org/rss/cs'
    response = requests.get(website_url)
    
    if response.status_code != 200: # validating urlrequest
        print('Request Not Succeed')
    else:
        pass
    
    soup = BeautifulSoup(response.text, "html.parser")
    submissions=soup.findAll('item') 
    
    titles = []
    descriptions = []
    authors = []
    urls = []
    
    for i in range(0, N):
        title_text = submissions[i].find('title').text
        title = title_text.split(' (arXiv')[0] #remove unecessray info
    
        desc_text = submissions[i].find('description').text
        desc_text = desc_text.replace('\n', '') # remove line break
        description = desc_text.strip('"<p>""</p>"') # remove <p></p>

        author_text = submissions[i].find('dc:creator').text
        author = re.sub(r'<[^<]+?>', '', author_text) #remove unecessray info

        url = submissions[i].get('rdf:about')
        url_request = requests.get(url)
        if url_request.status_code == 200: # validating urlrequest
            pass
        else:
            print(url+" site does not exist")

        descriptions.append(description)
        titles.append(title)
        authors.append(author)
        urls.append(url)
        
    df = pd.DataFrame({'titles':titles, 'descriptions':descriptions, 'authors':authors, 'urls':urls})
    df.to_csv('papers.csv')
    return df


In [31]:
papers = extract_paper(100)
papers.head(10)

,titles,descriptions,authors,urls
0,Computing the 2-adic complexity of two classes...,This paper contributes to compute 2-adic compl...,"Ming Yan, Tongjiang Yan, Yu Li",http://arxiv.org/abs/1912.06134
1,L3DOR: Lifelong 3D Object Recognition.,3D object recognition has been widely-applied....,"Yuyang Liu, Yang Cong, Gan Sun",http://arxiv.org/abs/1912.06135
2,Calibrated model-based evidential clustering u...,Evidential clustering is an approach to cluste...,Thierry Denoeux,http://arxiv.org/abs/1912.06137
3,ABOUT ML: Annotation and Benchmarking on Under...,"We present the ""Annotation and Benchmarking on...","Inioluwa Deborah Raji, Jingying Yang",http://arxiv.org/abs/1912.06166
4,Awareness in Practice: Tensions in Access to S...,Organizations cannot address demographic dispa...,"Miranda Bogen, Aaron Rieke, Shazeda Ahmed",http://arxiv.org/abs/1912.06171
5,Coevolution of Generative Adversarial Networks.,Generative adversarial networks (GAN) became a...,"Victor Costa, Nuno Louren&#xe7;o, Penousal Ma...",http://arxiv.org/abs/1912.06172
6,Training without training data: Improving the ...,Abbreviation disambiguation is important for a...,"Marta Skreta, Aryan Arbabi, Jixuan Wang, Mich...",http://arxiv.org/abs/1912.06174
7,Investigating the effectiveness of web adblock...,We investigate adblocking filters and the exte...,"Clayton Drazner, Nikola &#x110;uza, Hugo Jonk...",http://arxiv.org/abs/1912.06176
8,COEGAN: Evaluating the Coevolution Effect in G...,Generative adversarial networks (GAN) present ...,"Victor Costa, Nuno Louren&#xe7;o, Jo&#xe3;o C...",http://arxiv.org/abs/1912.06180
9,Learning Effective Visual Relationship Detecto...,We present our winning solution to the Open Im...,"Yichao Lu, Cheng Chang, Himanshu Rai, Guangwe...",http://arxiv.org/abs/1912.06185


## Part 2: SQL Knowledge

Final output: employeeid, firstname, lastname, address (the most recent), city, provinces, postalcode, moveindate (the most recent)

```
SELECT *
FROM employees e
LEFT JOIN (
SELECT employeeid, address, city, province, postalcode, moveindate
FROM addresses a
LEFT JOIN provinces p ON p.provinceid = a.provinceid
GROUP BY employeeid
HAVING moveindate = MAX(moveindate)
) AS temp ON e.employeeid = temp.employeeid
```